# trAIns AI vs Admiral AI - run experiments¶

Runs a series of experiments of OpenTTDLab, attempting to replicate the results from "trAIns: An Artificial Inteligence for OpenTTD" DOI 10.1109/SBGAMES.2009.15

Results are saved to 01_trains_ai_vs_admiral_ai_01_raw.csv.

In [1]:
!python -m pip install OpenTTDLab==0.0.72 pandas==2.2.0


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip3.11 install --upgrade pip


In [7]:
from openttdlab import run_experiments, bananas_ai, bananas_ai_library

def process_result(result):
    def get_company_value(player):
        try:
            return player['old_economy'][0]['company_value']
        except (KeyError, IndexError):
            return 0
    
    return (
        {
            'date': result['date'],
            'seed': result['experiment']['seed'],
            'terrain_type': result['chunks']['PATS']['0']['difficulty.terrain_type'],
            'name': \
                'Admiral AI' if player['name'].startswith('AdmiralAI') else \
                'trAIns AI' if player['name'].startswith('trAIns AI') else \
                'Unknown',
            'company_value': get_company_value(player),
            'money': player['money'],
        }
        for player in result['chunks']['PLYR'].values()
    )

results = run_experiments(
    openttd_version='13.4',
    opengfx_version='7.1',
    experiments=(
        {
            'seed': seed,
            'ais': (
                # To get a specific version of a library from BaNaNaS, we use the full MD5 rather than
                # the actual version number.
                # trAIns 2.1
                bananas_ai('54524149', 'trAIns', md5='c4c069dc797674e545411b59867ad0c2'),
                # AdmiralAI 25
                bananas_ai('41444d4c', 'AdmiralAI', md5='4ccd92fb8f8f01045145be99a28e14a6', ai_params=(
                    ('use_trains', '1'),
                    ('use_busses', '0'),
                    ('use_trucks', '0'),
                    ('use_planes', '0'),
                )),
            ),
            'days': 366 * 15 + 1,
            'openttd_config': f'''
                [difficulty]
                terrain_type={terrain_type}
                number_towns=2
                industry_density=2
                max_loan=300000
                initial_interest=3
                vehicle_costs=1
                subsidy_multiplier=1
                construction_cost=1
                economy=true
                quantity_sea_lakes=0
                vehicle_breakdowns=0
                town_council_tolerance=1
                disasters=true
                line_reverse_mode=true
                [game_creation]
                starting_year=1960
                max_x=512
                max_y=512
            ''',
        }
        for seed in range(0, 2)
        for terrain_type in [1, 3]
    ),
    result_processor=process_result,
)

Output()

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df.to_csv('01_trains_ai_vs_admiral_ai_results_01_raw.csv', index=False)